In [9]:
import cv2
import numpy as np

cap = cv2.VideoCapture(0)
_, frame = cap.read()
frame = cv2.flip(frame, 1)
prvs = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

while 1:
    _, frame = cap.read()
    frame = cv2.flip(frame, 1)
    current = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    winsize = 20

    flow = cv2.calcOpticalFlowFarneback(
        prev=prvs,
        next=current,
        flow=None,
        pyr_scale=0.3,
        levels=8,
        winsize=winsize,
        iterations=3,
        poly_n=5,
        poly_sigma=1.2,
        flags=0,
    )
    # More info on Farneback' method
    # http://www.diva-portal.org/smash/get/diva2:273847/FULLTEXT01.pdf

    flowim = np.zeros_like(frame)
    # flowim = frame.copy()
    step = winsize
    for y in range(0, flow.shape[0] - 1, step):
        for x in range(0, flow.shape[1] - 1, step):
            pt1 = (x, y)
            pt2 = (int(x + flow[y, x, 0] / 1.15), int(y + flow[y, x, 1] / 1.15))

            flowim = cv2.arrowedLine(flowim, pt1, pt2, (0, 255, 0), 1, tipLength=0.4)

    cv2.imshow("FlowArrow", flowim)
    if cv2.waitKey(1) & 0xFF == ord("q"):
        break
    prvs = current.copy()

cap.release()
cv2.destroyAllWindows()

In [11]:
import cv2

cap = cv2.VideoCapture(0)
_, frame = cap.read()
frame = cv2.flip(frame, 1)
prvs = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

while 1:
    _, frame = cap.read()
    frame = cv2.flip(frame, 1)
    current = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    flow = cv2.calcOpticalFlowFarneback(prvs, current, None, 0.3, 8, 15, 3, 5, 1.2, 0)

    flowim = frame.copy()
    step = 15
    for y in range(0, flow.shape[0] - 1, step):
        for x in range(0, flow.shape[1] - 1, step):
            pt1 = (x, y)
            pt2 = (int(x + flow[y, x, 0] / 2), int(y + flow[y, x, 1] / 2))

            if abs(flow[y, x, 0]) < 5 and abs(flow[y, x, 1]) < 5:
                flowim = cv2.arrowedLine(
                    flowim, pt1, pt2, (255, 255, 255), 1, tipLength=0.4
                )
            elif flow[y, x, 0] < 0 and abs(flow[y, x, 1]) <= abs(flow[y, x, 0]):
                flowim = cv2.arrowedLine(
                    flowim, pt1, pt2, (0, 0, 255), 2, tipLength=0.4
                )
            elif flow[y, x, 0] > 0 and abs(flow[y, x, 1]) <= abs(flow[y, x, 0]):
                flowim = cv2.arrowedLine(
                    flowim, pt1, pt2, (0, 255, 0), 2, tipLength=0.4
                )
            elif flow[y, x, 1] < 0:
                flowim = cv2.arrowedLine(
                    flowim, pt1, pt2, (0, 255, 255), 2, tipLength=0.4
                )
            elif flow[y, x, 1] > 0:
                flowim = cv2.arrowedLine(
                    flowim, pt1, pt2, (255, 100, 100), 2, tipLength=0.4
                )
            else:
                flowim = cv2.arrowedLine(
                    flowim, pt1, pt2, (255, 255, 255), 1, tipLength=0.4
                )

    cv2.imshow("FlowArrow", flowim)
    if cv2.waitKey(1) & 0xFF == ord("q"):
        break
    prvs = current.copy()

cap.release()
cv2.destroyAllWindows()

In [14]:
import cv2
import numpy as np

cap = cv2.VideoCapture(0)
_, frame = cap.read()
frame = cv2.flip(frame, 1)
prvs = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
flowhsv = np.zeros_like(frame)
flowhsv[..., 1] = 255


while 1:
    _, frame = cap.read()
    frame = cv2.flip(frame, 1)
    current = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    flow = cv2.calcOpticalFlowFarneback(prvs, current, None, 0.3, 8, 10, 3, 5, 1.2, 0)

    # cartesian to polar
    mag, ang = cv2.cartToPolar(flow[..., 0], flow[..., 1])
    flowhsv[..., 0] = ang * 180 / np.pi / 2
    # flowhsv[...,2] = 255
    # flowhsv[...,2] = 255*(mag > 5)
    # flowhsv[...,2] = mag*4
    # flowhsv[...,2] = cv2.normalize(mag,None,0,255,cv2.NORM_MINMAX)
    flowim = cv2.cvtColor(flowhsv, cv2.COLOR_HSV2BGR)
    cv2.imshow("FlowMagnitude", 4 * mag.astype("uint8"))
    cv2.imshow("FlowImage", flowim)
    # cv2.imshow('Frame', (flowim/2+frame/2).astype('uint8'))
    if cv2.waitKey(1) & 0xFF == ord("q"):
        break
    prvs = current.copy()

cap.release()
cv2.destroyAllWindows()

In [25]:
import cv2

# Download images from
# https://drive.google.com/file/d/1TbVn2K3Kxtntne19vPOQjf5pGs6Jo72H/view?usp=sharing

ref = cv2.imread("conan/conan1.jpg", cv2.COLOR_BGR2GRAY)
target = cv2.imread("conan/conan2.jpg", cv2.COLOR_BGR2GRAY)
h, w, _ = target.shape
target = cv2.resize(target, (int(w * 0.5), int(h * 0.5)))

detector = cv2.SIFT_create()
matcher = cv2.BFMatcher()

# detector = cv2.ORB_create()
# matcher = cv2.BFMatcher(cv2.NORM_HAMMING)

kp1, des1 = detector.detectAndCompute(ref, None)
kp2, des2 = detector.detectAndCompute(target, None)

print(str(len(kp1)) + "," + str(len(kp2)))
print(des1.shape)
print(des2.shape)
print(kp1[0].pt)
print(kp1[0].size)
print(kp1[0].angle)
print(des1[0])

matches = matcher.knnMatch(des1, des2, k=2)

# matches = sorted(matches, key = lambda x:x.distance)
good = []
for m, n in matches:
    if m.distance < 0.6 * n.distance:
        good.append(m)

result = cv2.drawMatches(ref, kp1, target, kp2, good, None, flags=2)

keypoint_ref = ref.copy()
cv2.drawKeypoints(ref, kp1, keypoint_ref, None, flags=4)
keypoint_target = target.copy()
cv2.drawKeypoints(target, kp2, keypoint_target, None, flags=4)

cv2.imshow("keypoint1", keypoint_ref)
cv2.imshow("keypoint2", keypoint_target)
cv2.imshow("match", result)

cv2.waitKey()

cv2.destroyAllWindows()

526,2956
(526, 128)
(2956, 128)
(7.328791618347168, 137.1427001953125)
1.9793020486831665
156.08792114257812
[  2.   2.   2.  10.  12.   5.   9.   5.  57.   9.   6.  10.   2.   1.
  13.  46. 127.  19.   1.   1.   4.   8.  19.  67.  30.  15.   3.   3.
  18.  55.  59.  28.  37.   4.   2.  18.  24.   3.   2.  28. 112.  18.
   2.   2.   1.   5.  73. 127. 113.  55.   9.   8.  10.  47. 107. 105.
  26.  63.  65.  37.  22.  54.  58.  14.  76.   7.   1.   6.  16.   4.
   9.  20. 127.  67.  11.   6.   1.   4.  10.  79.  56.  62.  44.  54.
  20.  53.  39.  29.  47.  72.  41.  12.   7.  70.  58.  32.  46.   4.
   1.   0.   0.   2.  63.  89. 115.  30.   6.   4.   8.  10.  47. 102.
  29.  57.  29.  33.  12.  57.  44.  15.  12.  38.  61.  53.   9.  17.
   9.   2.]


In [19]:
import numpy as np
import cv2

cap = cv2.VideoCapture(0)

detector = cv2.SIFT_create()
matcher = cv2.BFMatcher()

# detector = cv2.ORB_create()
# matcher = cv2.BFMatcher(cv2.NORM_HAMMING)

ref = cv2.imread("conan/conan1.jpg", cv2.COLOR_BGR2GRAY)
h, w, _ = ref.shape
ref = cv2.resize(ref, (int(w * 0.7), int(h * 0.7)))

kp1, des1 = detector.detectAndCompute(ref, None)

while True:
    _, target = cap.read()

    kp2, des2 = detector.detectAndCompute(target, None)

    matches = matcher.match(des1, des2)

    matches = sorted(matches, key=lambda x: x.distance)

    result = cv2.drawMatches(ref, kp1, target, kp2, matches[:50], None, flags=2)

    cv2.imshow("result", result)

    if cv2.waitKey(1) & 0xFF == ord("q"):
        break

cap.release()
cv2.destroyAllWindows()

In [21]:
import numpy as np
import cv2

cap = cv2.VideoCapture(0)

detector = cv2.SIFT_create()
matcher = cv2.BFMatcher()

# detector = cv2.ORB_create()
# matcher = cv2.BFMatcher(cv2.NORM_HAMMING)

ref = cv2.imread("conan/conan1.jpg", cv2.COLOR_BGR2GRAY)
h, w, _ = ref.shape
ref = cv2.resize(ref, (int(w * 1.0), int(h * 1.0)))

kp1, des1 = detector.detectAndCompute(ref, None)

while True:
    _, target = cap.read()

    kp2, des2 = detector.detectAndCompute(target, None)

    matches = matcher.knnMatch(des1, des2, k=2)  # Relative Matching

    good = []
    for m, n in matches:
        if m.distance < 0.6 * n.distance:
            good.append(m)

    result = cv2.drawMatches(ref, kp1, target, kp2, good, None, flags=2)

    cv2.imshow("result", result)

    if cv2.waitKey(1) & 0xFF == ord("q"):
        break

cap.release()
cv2.destroyAllWindows()

In [9]:
import numpy as np
import cv2
import time

cap = cv2.VideoCapture(0)

detector = cv2.SIFT_create()
matcher = cv2.BFMatcher()

# detector = cv2.ORB_create()
# matcher = cv2.BFMatcher(cv2.NORM_HAMMING)

ref1 = cv2.imread("tasks_s/1.jpg", cv2.COLOR_BGR2GRAY)
h, w, _ = ref1.shape
ref1 = cv2.resize(ref1, (int(w * 0.2), int(h * 0.2)))

ref2 = cv2.imread("tasks_s/2.jpg", cv2.COLOR_BGR2GRAY)
h, w, _ = ref2.shape
ref2 = cv2.resize(ref2, (int(w * 0.2), int(h * 0.2)))

kp1, des1 = detector.detectAndCompute(ref1, None)
kp2, des2 = detector.detectAndCompute(ref2, None)

prev_frame_time = 0
new_frame_time = 0

while True:
    _, target = cap.read()

    kp_c, des_c = detector.detectAndCompute(target, None)

    matches1 = matcher.knnMatch(des1, des_c, k=2)  # Relative Matching
    matches2 = matcher.knnMatch(des2, des_c, k=2)  # Relative Matching

    good1 = list(
        map(
            lambda x: x[0],
            filter(lambda x: x[0].distance < 0.6 * x[1].distance, matches1),
        )
    )
    good2 = list(
        map(
            lambda x: x[0],
            filter(lambda x: x[0].distance < 0.6 * x[1].distance, matches2),
        )
    )

    result1 = cv2.drawMatches(ref1, kp1, target, kp_c, good1, None, flags=2)
    result2 = cv2.drawMatches(ref2, kp2, target, kp_c, good2, None, flags=2)

    font = cv2.FONT_HERSHEY_SIMPLEX
    new_frame_time = time.time()
    fps = 1 / (new_frame_time - prev_frame_time)
    prev_frame_time = new_frame_time
    fps = int(fps)
    fps = str(fps)
    cv2.putText(result1, fps, (7, 70), font, 3, (100, 255, 0), 3, cv2.LINE_AA)

    cv2.imshow("result1", result1)
    cv2.imshow("result2", result2)

    if cv2.waitKey(1) & 0xFF == ord("q"):
        break

cap.release()
cv2.destroyAllWindows()